In [129]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

tweet_id - int - index \
username - string\
text - string\
created_at - datetime\
location - string\
source - string\
url - string

text - split each sentence into individual words, without whitespace, puncutation, all lowercase.\
Each value of text is a list ['i', 'like', 'coffee']\
create word frequency dictionary ^

In [131]:
tw_mitaka = pd.read_csv("tw_mitaka1.csv")
tw_mitaka['created_at'] =  pd.to_datetime(tw_mitaka['created_at'])

In [153]:
#VADER DF has no modifications on tweet text
tw_mitaka_vader = pd.read_csv("tw_mitaka1.csv")

In [133]:
tw_mitaka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    939 non-null    int64              
 1   username    939 non-null    object             
 2   text        939 non-null    object             
 3   created_at  939 non-null    datetime64[ns, UTC]
 4   location    633 non-null    object             
 5   source      939 non-null    object             
 6   url         939 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 51.5+ KB


In [134]:
tw_mitaka.head()

,tweet_id,username,text,created_at,location,source,url
0,1553777739773186048,Vulpeqxla,i need to draw mitaka again soon or else i wil...,2022-07-31 16:20:55+00:00,they/them ♥,Twitter for iPhone,https://twitter.com/Vulpeqxla/status/155377773...
1,1553776451186458625,aokisotaa,finally decided to check out the denji edit ta...,2022-07-31 16:15:48+00:00,"19, they/she",Twitter for Android,https://twitter.com/aokisotaa/status/155377645...
2,1553775430276194304,retromanjapan,The Showa Retro In https://t.co/H3BK93ITy6 #wo...,2022-07-31 16:11:44+00:00,"Tokyo, Japan",Revive Social App,https://twitter.com/retromanjapan/status/15537...
3,1553773879650394115,nymphahri,Asa Mitaka ⛓️\nShould I make the actual cospla...,2022-07-31 16:05:35+00:00,Italia,Twitter for Android,https://twitter.com/nymphahri/status/155377387...
4,1553756835047690240,moonrcin,mitaka best girl \n\n#チェンソーマン #chainsawman htt...,2022-07-31 14:57:51+00:00,🇵🇭,Twitter for iPhone,https://twitter.com/moonrcin/status/1553756835...


In [135]:
#Stop words, links, emotes, non-ascii characters are removed.
stops = set(stopwords.words('english'))

for i in range(len(tw_mitaka['text'])):
    tweet_text = []
    #https://stackoverflow.com/a/24399874
    tw_mitaka['text'][i] = re.sub(r'[^\x00-\x7f]',r'', tw_mitaka['text'][i])
    #https://stackoverflow.com/a/65330690
    tw_mitaka['text'][i] = re.sub(r"http\S+", "", tw_mitaka['text'][i])
    
    words = tw_mitaka['text'][i].lower().split()
    words = [''.join(letter for letter in word if letter not in (string.punctuation)) for word in words]
    for word in words:
        if word not in stops:
            tweet_text.append(word)
    tw_mitaka['text'][i] = tweet_text

In [136]:
print(tw_mitaka['text'][25])

['showa', 'retro', 'mitaka', 'tokyo', '16', 'row', 'house', 'announced', 'therowhousestructure', 'thejrjapanrailwaychuoline']


In [137]:
#Word frequency dict
word_freq = {}

for text_list in tw_mitaka['text']:
    for word in text_list:
        if word not in word_freq:
            word_freq[word] = 0
        word_freq[word] += 1

In [138]:
dict(list(word_freq.items())[0:5])

{'need': 18, 'draw': 8, 'mitaka': 413, 'soon': 9, 'else': 8}

In [139]:
#Creating dataframe from word freq dict
df_wordfreq = pd.DataFrame.from_dict(word_freq, orient='index', columns=['Freq'])
df_wordfreq = df_wordfreq.drop('')

In [140]:
df_wordfreq.sort_values('Freq', ascending=False).head(5)

,Freq
devil,524
war,524
mitaka,413
chainsawman,223
asa,213


In [151]:
#Sentimental analysis function with VADER
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05 :
        return("Positive")
    elif sentiment_dict['compound'] <= -0.05 :
        return("Negative")
    else :
        return("Neutral")

In [154]:
#Adding score column for sentiment analysis
tw_mitaka_vader = tw_mitaka_vader.reindex(columns = tw_mitaka_vader.columns.tolist() + ['score'])

In [155]:
#VADER Sentimental analysis on text. 
for i in range(len(tw_mitaka_vader['text'])):
    tw_mitaka_vader['score'] = sentiment_scores(tw_mitaka_vader['text'][i])

In [156]:
tw_mitaka_vader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_id    939 non-null    int64 
 1   username    939 non-null    object
 2   text        939 non-null    object
 3   created_at  939 non-null    object
 4   location    633 non-null    object
 5   source      939 non-null    object
 6   url         939 non-null    object
 7   score       939 non-null    object
dtypes: int64(1), object(7)
memory usage: 58.8+ KB


In [157]:
print(tw_mitaka_vader.groupby('score').size())

score
Negative    939
dtype: int64


In [ ]:
#Not sure why only negatives, gotta look into that.
#Next query will be on Sunday August 14th. 